# Load Modules

In [1]:
from collections import defaultdict

import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt
import numpy as np
import optax
import sklearn.linear_model
import tqdm
from flax import nnx
from sklearn.datasets import load_diabetes
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Data Loading

In [2]:
data = load_diabetes()
# print(data["DESCR"])

X, y = data["data"], data["target"].reshape(-1, 1)
X, y = StandardScaler().fit_transform(X), StandardScaler().fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

X_train.shape, X_test.shape

((353, 10), (89, 10))

# Modeling

In [3]:
class LinearRegressor(nnx.Module):
    def __init__(self, din: int, rngs: nnx.Rngs):
        self.linear = nnx.Linear(din, 1, rngs=rngs)

    def __call__(self, x: jax.Array):
        y = self.linear(x)
        return y

# Training

In [4]:
@nnx.jit
def train_step(model, optimizer, x, y):
    def loss_fn(model):
        y_pred = model(x)
        return jnp.mean((y_pred - y) ** 2)

    loss, grads = nnx.value_and_grad(loss_fn)(model)
    optimizer.update(grads)

    return loss

In [5]:
def train_loop(model, optimizer, X, y, n_epochs=32, n_batch_size=32):
    all_ilocs = np.arange(len(X))
    n_split = len(all_ilocs) // n_batch_size

    with tqdm.trange(1, n_epochs + 1) as progress_bar:
        for epoch in progress_bar:
            ilocs_batches = jnp.array_split(
                jax.random.permutation(jax.random.PRNGKey(epoch), all_ilocs),
                n_split,
            )

            batch_loss = np.zeros(len(ilocs_batches))
            for i, _ilocs in enumerate(ilocs_batches):
                batch_loss[i] = train_step(model, optimizer, X[_ilocs], y[_ilocs])

            progress_bar.set_postfix(dict(epoch_loss=f"{batch_loss.mean():.3f}"))

In [6]:
model = LinearRegressor(X_train.shape[1], rngs=nnx.Rngs(0))
optimizer = nnx.Optimizer(model, optax.adam(1e-3))

train_loop(model, optimizer, X_train, y_train, n_epochs=1000, n_batch_size=128)

100%|██████████████████████████████████████████████████████████| 1000/1000 [00:02<00:00, 342.78it/s, epoch_loss=0.473]


# Evaluation

{'flax': 1.447333582230181, 'sklearn': 1.447333582230181}


In [7]:
lr = sklearn.linear_model.LinearRegression().fit(X_train, y_train)

plt.plot(y_test, label="y_true", alpha=0.5)
plt.plot(model(X_test), label="Flax", alpha=0.5)
plt.plot(lr.predict(X_test), label="sklearn", alpha=0.5)
plt.legend()

metrics = dict(
    flax=mean_absolute_percentage_error(y_test, model(X_test).ravel()),
    sklearn=mean_absolute_percentage_error(y_test, model(X_test)),
)

print(metrics)